In [1]:
from powersimdata.scaling.clean_capacity_scaling.auto_capacity_scaling import IndependentStrategyManager, TargetManager, Resource
import pandas as pd
import os

In [ ]:
os.getcwd()

In [ ]:
target_data = pd.read_excel(os.path.relpath(os.path.join('tests','test_bulk_input.xlsx')))
target_data

In [ ]:
strategy_manager = IndependentStrategyManager()

In [ ]:
strategy_manager.targets_from_data_frame(target_data)

In [ ]:
print(strategy_manager.targets['Pacific'])

In [ ]:
print(strategy_manager.targets['Atlantic4'])

In [2]:
western = pd.read_excel('Capacity_Scaling_Western_Test_Case.xlsx')
western

,State,coal_generation,geothermal_generation,hydro_generation,ng_generation,nuclear_generation,solar_generation,wind_generation,coal_capacity,geothermal_capacity,...,solar_percentage,geothermal_counts,hydro_counts,nuclear_counts,solar_counts,wind_counts,solar_added_capacity_independent,wind_added_capacity_independent,solar_added_capacity_collaborative,wind_added_capacity_collaborative
0,Arizona,2.936274e+07,0.0,7.251077e+06,3.046459e+07,36980640.0,3.919427e+06,6.476678e+05,6244.8,0.0,...,0.858,no,no,no,yes,yes,6022.355,851.958,5579.819,789.090
1,California,3.155553e+05,11612448.0,2.970441e+07,9.166335e+07,20405232.0,2.016247e+07,1.717961e+07,62.5,1322.0,...,0.540,yes,yes,no,yes,yes,42211.767,24272.718,29122.253,16740.337
2,Colorado,3.052065e+07,0.0,1.902903e+06,1.158108e+07,0.0,8.441691e+05,1.019488e+07,5524.7,0.0,...,0.076,no,no,no,yes,yes,717.783,5180.735,1238.516,8936.229
3,Idaho,0.000000e+00,0.0,9.033229e+06,2.695616e+06,0.0,1.873226e+05,2.318389e+06,10.2,0.0,...,0.075,no,no,no,yes,yes,120.000,974.400,354.368,2876.503
4,Montana Western,1.440111e+07,0.0,1.008253e+07,4.543809e+05,0.0,0.000000e+00,2.283680e+06,2433.8,0.0,...,0.000,no,no,no,yes,yes,0.000,657.300,0.000,1940.400
5,Nevada,3.647902e+06,3364272.0,1.789219e+06,2.485498e+07,0.0,3.206166e+06,4.775214e+05,1103.8,383.0,...,0.870,no,no,no,yes,yes,7627.010,715.793,4719.886,442.811
6,New Mexico Western,1.840395e+07,0.0,1.479720e+05,4.605579e+06,0.0,6.692370e+05,1.889408e+06,3741.2,0.0,...,0.262,no,no,no,yes,yes,1172.987,2363.258,921.947,1856.856
7,Oregon,1.682094e+06,0.0,3.454937e+07,1.472753e+07,0.0,1.502388e+05,7.359174e+06,642.2,0.0,...,0.020,no,no,no,yes,yes,182.317,5769.165,295.011,9332.102
8,Utah,3.092791e+07,0.0,7.595420e+05,1.257747e+07,0.0,1.620673e+06,1.178095e+06,4894.1,0.0,...,0.579,no,no,no,yes,yes,2122.988,963.010,2527.234,1145.996
9,Washington,5.894132e+06,0.0,7.834801e+07,7.842268e+06,10540800.0,1.357497e+03,7.239663e+06,1459.8,0.0,...,0.000,no,yes,yes,yes,yes,1.000,3074.000,2.953,9074.681


In [3]:
resources_dict = {
    'coal': {'prev_generation':'coal_generation', 'prev_capacity': 'coal_capacity'},
    'geothermal':{'prev_generation':'geothermal_generation', 'prev_capacity':'geothermal_capacity'},
    'ng': {'prev_generation':'ng_generation','prev_capacity':'ng_capacity'},
    'nuclear':{'prev_generation':'nuclear_generation','prev_capacity':'nuclear_capacity'},
    'hydro':{'prev_generation':'hydro_generation','prev_capacity':'hydro_capacity'},
    'solar':{'prev_generation':'solar_generation', 'prev_capacity':'solar_capacity',
             'no_congestion_cap_factor':'no_cong_solar_cf','prev_cap_factor':'prev_sim_solar_cf'},
    'wind':{'prev_generation':'wind_generation', 'prev_capacity':'wind_capacity',
            'no_congestion_cap_factor':'no_cong_wind_cf','prev_cap_factor':'prev_sim_wind_cf'}
}
resources_dict

{'coal': {'prev_generation': 'coal_generation',
  'prev_capacity': 'coal_capacity'},
 'geothermal': {'prev_generation': 'geothermal_generation',
  'prev_capacity': 'geothermal_capacity'},
 'ng': {'prev_generation': 'ng_generation', 'prev_capacity': 'ng_capacity'},
 'nuclear': {'prev_generation': 'nuclear_generation',
  'prev_capacity': 'nuclear_capacity'},
 'hydro': {'prev_generation': 'hydro_generation',
  'prev_capacity': 'hydro_capacity'},
 'solar': {'prev_generation': 'solar_generation',
  'prev_capacity': 'solar_capacity',
  'no_congestion_cap_factor': 'no_cong_solar_cf',
  'prev_cap_factor': 'prev_sim_solar_cf'},
 'wind': {'prev_generation': 'wind_generation',
  'prev_capacity': 'wind_capacity',
  'no_congestion_cap_factor': 'no_cong_wind_cf',
  'prev_cap_factor': 'prev_sim_wind_cf'}}

In [4]:
strategy_manager = IndependentStrategyManager()

for row in western.itertuples():
    target = TargetManager(row.State, 
                           row.target_2030, 
                           'CE category', 
                           row.demand_2030,
                           row.external_count, 
                           row.solar_percentage)
    
    allowed_resources = []
    if row.geothermal_counts == 'yes':
        allowed_resources.append('geothermal')
    if row.hydro_counts == 'yes':
        allowed_resources.append('hydro')
    if row.nuclear_counts == 'yes':
        allowed_resources.append('nuclear')
    if row.solar_counts == 'yes':
        allowed_resources.append('solar')
    if row.wind_counts == 'yes':
        allowed_resources.append('wind')
    target.set_allowed_resources(allowed_resources)

    
    for res, mapping in resources_dict.items():
        resource = Resource(res,1)

        if res == 'solar' or res == 'wind':
            resource.set_capacity(
                getattr(row,mapping['no_congestion_cap_factor']), 
                getattr(row,mapping['prev_capacity']), 
                getattr(row,mapping['prev_cap_factor'])
            )
        else:
            resource.set_capacity(
                None, 
                getattr(row,mapping['prev_capacity']), 
                None
            )
            
        resource.set_generation(getattr(row,mapping['prev_generation']))
        target.add_resource(resource)
        
    strategy_manager.add_target(target)

In [5]:
strategy_manager.data_frame_of_next_capacities()

,region_name,next_solar_capacity,next_wind_capacity
0,Arizona,4.035621e+06,6.679000e+05
1,California,2.648424e+07,2.256065e+07
2,Colorado,1.828196e+05,2.222701e+06
3,Idaho,0.000000e+00,0.000000e+00
4,Montana Western,0.000000e+00,0.000000e+00
5,Nevada,6.954556e+06,1.039187e+06
6,New Mexico Western,6.652292e+05,1.873814e+06
7,Oregon,5.360365e+04,2.626579e+06
8,Utah,1.010676e+06,7.348785e+05
9,Washington,0.000000e+00,0.000000e+00


In [6]:
print(strategy_manager.targets['Arizona'])

{
    "allowed_resources": [
        "solar",
        "wind"
    ],
    "ce_category": "CE category",
    "ce_target": 14556583.0284,
    "ce_target_fraction": 0.15,
    "external_ce_historical_amount": 0.0,
    "py/object": "powersimdata.scaling.clean_capacity_scaling.auto_capacity_scaling.TargetManager",
    "region_name": "Arizona",
    "resources": {
        "coal": {
            "addl_curtailment": 0,
            "name": "coal",
            "no_congestion_cap_factor": null,
            "prev_cap_factor": null,
            "prev_capacity": 6244.8,
            "prev_curtailment": null,
            "prev_generation": 29362742.087,
            "prev_scenario_num": 1,
            "py/object": "powersimdata.scaling.clean_capacity_scaling.auto_capacity_scaling.Resource"
        },
        "geothermal": {
            "addl_curtailment": 0,
            "name": "geothermal",
            "no_congestion_cap_factor": null,
            "prev_cap_factor": null,
            "prev_capacity": 0.0,


In [7]:
strategy_manager.targets['Arizona'].calculate_added_capacity()

(4035620.986450091, 667899.9767784534)